In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor


df = pd.read_csv('soil_data.csv')


X = df[['N', 'P', 'K', 'pH', 'Moisture', 'Temperature']]
y_crop = df['Recommended Crop']
y_fertilizer = df['Recommended Fertilizer & Quantity']
y_yield = df['Predicted Yield']


preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['N', 'P', 'K', 'pH', 'Moisture', 'Temperature'])
    ]
)


model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', MultiOutputRegressor(RandomForestRegressor()))
])


y_crop_encoded = pd.get_dummies(y_crop)
y_fertilizer_encoded = pd.get_dummies(y_fertilizer)


y_combined = pd.concat([y_crop_encoded, y_fertilizer_encoded, y_yield], axis=1)


X_train, X_test, y_train, y_test = train_test_split(X, y_combined, test_size=0.2, random_state=42)


model.fit(X_train, y_train)


predictions = model.predict(X_test)


import joblib
joblib.dump(model, 'soil_model.pkl')


['soil_model.pkl']